This is simple solution for the example case, where only 1 source and 1 gateway node was considered.

In [26]:
import math

# 1st Input line
input_line1 = input("Enter N, k, m, and |E| separated by space: ")

# Split the input line into separate values
N, k, m, total_edges = map(int, input_line1.split())


# 2nd Input line
# Split the input line into separate values
S = []
D = []
for i in range(k):
    input_line2 = input(f"Enter source node {i + 1} ID and requested bandwidth separated by space: ")
    source_id, bandwidth = map(int, input_line2.split())
    S.append(source_id)
    D.append(bandwidth)

# 3rd Input line
input_line3 = input("Enter gateway nodes IDs separated by space: ")
G = list(map(int, input_line3.split()))


# Take input for each edge
edges = []
E = []
for i in range(total_edges):
    edge_input = input(f"Enter edge {i} information (Edge ID, starting node, ending node, capacity, power) separated by space: ")
    edge_info = list(map(int, edge_input.split()))
    edges.append(edge_info)

E = [sublist[1:] for sublist in edges]
# Last input line the constraits floating point numbers representing the coefficients (α, β, δ)
input_line4 = input("Enter the coefficients (α, β, δ) separated by space: ")
coef = list(map(float, input_line4.split()))
# Print the values for verification
print("Number of nodes:", N)
print("Number of Sources:", k)
print("Number of Gateways:", m)
print("Number of Edges:", total_edges)
print("Source IDs:", S)
print("Bandwidths:", D)
print("Gateway IDs: ", G)
print("Edges information:")
i = 0
for edge in edges:
    print(f"Edge ID: {edge[0]}, Start Node ID: {edge[1]}, End Node ID: {edge[2]}, Capacity: {edge[3]}, Power: {edge[4]}")

print("α, β, δ Coefficients: ", coef)

Enter N, k, m, and |E| separated by space: 4 1 1 6
Enter source node 1 ID and requested bandwidth separated by space: 0 10
Enter gateway nodes IDs separated by space: 3
Enter edge 0 information (Edge ID, starting node, ending node, capacity, power) separated by space: 0 0 1 10 1
Enter edge 1 information (Edge ID, starting node, ending node, capacity, power) separated by space: 1 0 1 5 3
Enter edge 2 information (Edge ID, starting node, ending node, capacity, power) separated by space: 2 0 2 15 2
Enter edge 3 information (Edge ID, starting node, ending node, capacity, power) separated by space: 3 1 3 10 2
Enter edge 4 information (Edge ID, starting node, ending node, capacity, power) separated by space: 4 1 2 20 1
Enter edge 5 information (Edge ID, starting node, ending node, capacity, power) separated by space: 5 2 3 10 3
Enter the coefficients (α, β, δ) separated by space: 0.33 0.33 0.33
Number of nodes: 4
Number of Sources: 1
Number of Gateways: 1
Number of Edges: 6
Source IDs: [0]
B

In [27]:
def find_alternative_paths(graph, start, end, path=[]):
    if start == end:
        return [path]
    if start not in graph:
        return []

    paths = []
    for edge_id, node in graph[start]:
        if edge_id not in path:
            new_paths = find_alternative_paths(graph, node, end, path + [edge_id])
            paths.extend(new_paths)
    return paths

edges = []
for i, sublist in enumerate(E):
    edges.extend([i, sublist[0:2]])
def edges_to_graph(edges):
    graph = {}
    for i in range(0, len(edges), 2):
        edge_id, connection = edges[i], edges[i+1]
        if connection[0] not in graph:
            graph[connection[0]] = []
        graph[connection[0]].append((edge_id, connection[1]))
    return graph

graph = edges_to_graph(edges)
source_node = S[0]
gateway_node = G[0]
paths = find_alternative_paths(graph, source_node, gateway_node)
#print(paths)

def calculate_data_flow(E, paths, bandwidth):
    data = bandwidth
    data_flow_results = []
    power_consumptions = []
    capacities = []
    data_flow_min_results = []
    network_utils = []
    energy_efficiency = []
    for path_indexes in paths:
        edge_util = 0
        data_flow = 0
        data_flow_min = 1000
        capacity = 0
        cap = 0
        power_consumption = 0
        power_consumption_edge = 0
        energy_efficiency_edge = 0
        for i in range(len(path_indexes)):
            power_consumption_edge = 0
            if E[path_indexes[i]][2] >= data:
              capacity = E[path_indexes[i]][2]
              cap += capacity
              power_consumption += E[path_indexes[i]][3]
              power_consumption_edge = E[path_indexes[i]][3]
              data_flow += data
              data_flow_min = min(data_flow_min,E[path_indexes[i]][2])
              edge_util += data/capacity
              network_util = edge_util
              energy_efficiency_edge += ((power_consumption_edge/capacity))
            else:
              capacity = E[path_indexes[i]][2]
              cap += capacity
              power_consumption += E[path_indexes[i]][3]
              data_flow += 0
              data_flow_min = min(data_flow_min,E[path_indexes[i]][2])
              power_consumption_edge = E[path_indexes[i]][3]
              edge_util += capacity/data
              network_util = edge_util #/(len(path_indexes))
              energy_efficiency_edge += ((power_consumption_edge/capacity))
        data_flow_results.append(data_flow)
        data_flow_min_results.append(data_flow_min)
        power_consumptions.append(power_consumption)
        capacities.append(cap)
        network_utils.append(network_util)
        energy_efficiency.append(energy_efficiency_edge)
    return data_flow_results, power_consumptions, capacities, data_flow_min_results, network_utils, energy_efficiency

bandwidth = D[0]
data_flow_results, power_consumptions,  capacities, data_flow_min_results, network_utils, energy_efficiency= calculate_data_flow(E, paths, bandwidth)
#print(E)
#print(data_flow_results, power_consumptions,  capacities, data_flow_min_results, network_utils, energy_efficiency)

[20, 30, 10, 20, 20] [3, 5, 5, 7, 5] [20, 40, 15, 35, 25] [10, 10, 5, 5, 10] [2.0, 2.5, 1.5, 2.0, 1.6666666666666665] [0.30000000000000004, 0.45, 0.8, 0.95, 0.43333333333333335]


In [28]:

# Generate Ds dictionary with example data
Ds = {s: {g: D[s-1] for g in G} for s in S}
# Network thoughtput, the total data successfully delivered to the gateways
T  = sum(Ds[s][g] for s in S for g in G)
net_thoughtput = [data_flow_min_results[i]/T for i in range(len(data_flow_min_results))]
#print("\nPath throughtput T_norm:", net_thoughtput)

# Energy consumption, the total energy consumption in the network for transmitting data:
#energy_consumption = sum([p_val * 1 / c_val * f_val for f_val, p_val, c_val in zip(data_flow_results, power_consumptions, capacities)])
#print("\nTotal network energy consumption: ", energy_consumpt
# Energy efficiency
energy = [data_flow_min_results[k]*energy_efficiency[k] for k in range(len(data_flow_min_results))]
eta = [data_flow_min_results[k]/energy[k] for k in range(len(data_flow_min_results))]

#n_norm = [(eta[l]-min(eta))/(max(eta) - min(eta)) for l in range(len(eta))]
#print(eta)
#eta = T/energy_consumption
#print("\nTotal network energy efficiency: ", eta)

# Overall Network Utilization
U = sum(network_utils)
print("\nAll possible paths: ", paths)
# Global objective function
#print("\Energy efficiency: ", eta)
#print("\nNetwork thoughput: ", data_flow_min_results)
#print("\nNetwork utilization", network_utils)
G_score = [coef[0] * eta[i] + coef[1] *data_flow_min_results[i] + coef[2] * network_utils[i]/total_edges for i in range(len(network_utils))]

# Output
max_value = max(G_score)
max_indexes = [index for index, value in enumerate(G_score) if value == max_value]
index_max = G_score.index(max_value)
output = [paths[i] for i in max_indexes]

print("\nExpected output is: ")
expected_output = []
for i in range(len(output)+1):
    expected_output.append(output[0][i])

expected_output.append(data_flow_min_results[index_max])

print(*expected_output)
print("The optimum path with Edge IDs is: ", *output[0])
print("The path data flow is: ", data_flow_min_results[index_max])



All possible paths:  [[0, 3], [0, 4, 5], [1, 3], [1, 4, 5], [2, 5]]
\Energy efficiency:  [3.333333333333333, 2.2222222222222223, 1.25, 1.0526315789473684, 2.3076923076923075]

Network thoughput:  [10, 10, 5, 5, 10]

Network utilization [2.0, 2.5, 1.5, 2.0, 1.6666666666666665]

Expected output is: 
0 3 10
The optimum path with Edge IDs is:  0 3
The path data flow is:  10
